In [10]:
import os
import nltk
from allennlp.commands.elmo import ElmoEmbedder
import numpy as np
import torch
import pickle
import time

In [5]:
elmo = ElmoEmbedder()

02/12/2019 14:28:50 - INFO - allennlp.commands.elmo -   Initializing ELMo.


In [3]:
print(os.getcwd())

D:\Academic & Professional\Academic\Master Thesis AI\Master-Thesis\Code\Notebooks


In [2]:
path_to_MLI_data = "../datasets/Data/MultiNLI/"

train          = path_to_MLI_data + "multinli_1.0_train.txt"
dev_matched    = path_to_MLI_data + "multinli_1.0_dev_matched.txt"
dev_mismatched = path_to_MLI_data + "multinli_1.0_dev_mismatched.txt"

path_to_test = "test.txt"

In [16]:
a = np.random.rand(1).astype(np.float32)
dtype = a.dtype
print(dtype)
np.dtype(eval('np.' + str(dtype))).itemsize

float32


4

In [6]:
class Variable_array_storage(object):

    def __init__(self, storage_file_path, num_data_elements_per_line, write_mode, dtype):

        self._storage_file_path          = open(storage_file_path + '.mem', write_mode)
        self._cursor_positions_file_path = storage_file_path + '.idx'

        self._num_data_elements_per_line = num_data_elements_per_line # Load into memory

        self._write_mode = write_mode

        
        if (write_mode == 'rb'):
            with open(self._cursor_positions_file_path, 'rb') as f:
                self._instances_positions = pickle.load(f)
            self._data_num_bytes = np.dtype(eval('np.' + str(self._instances_positions[0]))).itemsize
        else:
            self._instances_positions = [dtype]
            self._data_num_bytes = np.dtype(eval('np.' + str(dtype))).itemsize

        



    def save_line_sequential(self, sentences_encodings):
        if (self._num_data_elements_per_line > 1):
            for instance_num in range(self._num_data_elements_per_line):
                self._save_instance_sequential(sentences_encodings[instance_num])
        else:
            self._save_instance_sequential(sentences_encodings)



    def _save_instance_sequential(self, sentence_encoding):
        self._instances_positions.append(self._storage_file_path.tell())
        for seq_ele in range(sentence_encoding.shape[1]):
            for encoding_type in range(self._num_encodings_types):# TODO: This variable can be moved to parent class
                for value_idx in range(self._size_encodings):
                    self._storage_file_path.write(sentence_encoding[encoding_type][seq_ele][value_idx])



    def save_instances_positions(self):
        with open(self._cursor_positions_file_path, 'wb+') as f:
            pickle.dump(self._instances_positions, f)



    def close_file(self):
        self._storage_file_path.close()



    def load_line_at_index(self, idx):
        raise NotImplemented
        

        

class Variable_ELMO_array_storage(Variable_array_storage):

    def __init__(self, storage_file_path, num_data_elements_per_line, write_mode, dtype_size_bytes=None, num_encodings_types=3, size_encodings=1024):
        super(Variable_ELMO_array_storage, self).__init__(storage_file_path, num_data_elements_per_line, write_mode, dtype_size_bytes)

        # ELMO embeddings characteristics
        self._num_encodings_types = num_encodings_types
        self._size_encodings      = size_encodings



    def load_line_at_index(self, line_idx):
        dtype = eval('np.' + str(self._instances_positions[0]))
        line_idx = 1 + line_idx*self._num_data_elements_per_line
        self._storage_file_path.seek(self._instances_positions[line_idx])
        line = []
        for instance_num in range(self._num_data_elements_per_line):
            if (line_idx + instance_num < len(self._instances_positions)):
                if (line_idx + instance_num + 1 < len(self._instances_positions)):
                    seq_length = self._instances_positions[line_idx + instance_num + 1] - self._instances_positions[line_idx + instance_num]
                else:
                    self._storage_file_path.seek(-1,2)
                    last_position = self._storage_file_path.tell() + 1 # +1 to account for the actual position the next instance would be at
                    self._storage_file_path.seek(self._instances_positions[line_idx + instance_num]) # Restore cursor position to the beginning of the instance
                    seq_length = last_position - self._instances_positions[line_idx + instance_num]
                seq_length = int(seq_length/(self._num_encodings_types * self._size_encodings * self._data_num_bytes))
            else:
                raise IndexError("There is no such line number (" + str(line_idx) + ") in file " + self._storage_file_path.name + ".")

            data_instance = np.empty([self._num_encodings_types, seq_length, self._size_encodings], dtype=dtype)

            for seq_ele in range(seq_length):
                for encoding_type in range(self._num_encodings_types):
                    for value_idx in range(self._size_encodings):
                        data_instance[encoding_type][seq_ele][value_idx] = np.frombuffer(self._storage_file_path.read(self._data_num_bytes), dtype=dtype)

            line.append(data_instance)

        return line

In [106]:
test_file_name = "test"
a = [[np.random.rand(3, 2, 2), np.random.rand(3, 4, 2)], [np.random.rand(3, 3, 2), np.random.rand(3, 5, 2)]]
print(a[0][0].dtype)
print(a[0][1].dtype)
print(a[1][0].dtype)
print(a[1][1].dtype)
print(a)

float64
float64
float64
float64
[[array([[[0.00108573, 0.29183447],
        [0.77035459, 0.90411301]],

       [[0.26867345, 0.12984343],
        [0.04426825, 0.89142792]],

       [[0.91958899, 0.29741063],
        [0.72152843, 0.54500064]]]), array([[[0.60545101, 0.77043845],
        [0.11846807, 0.32999093],
        [0.98382971, 0.36558403],
        [0.10523205, 0.46761935]],

       [[0.03855612, 0.31502763],
        [0.24207041, 0.73431365],
        [0.10535907, 0.39135685],
        [0.2826739 , 0.91535291]],

       [[0.00169617, 0.97615194],
        [0.96185165, 0.2999511 ],
        [0.9812867 , 0.10195023],
        [0.67164552, 0.01241636]]])], [array([[[0.41212383, 0.96545007],
        [0.0546776 , 0.7107712 ],
        [0.5505881 , 0.556125  ]],

       [[0.19634092, 0.6540999 ],
        [0.17051387, 0.61365517],
        [0.24236891, 0.17776813]],

       [[0.15770505, 0.56735433],
        [0.19640796, 0.84603931],
        [0.66754472, 0.68189391]]]), array([[[0.52480888, 0.25

In [107]:
memmap = Variable_ELMO_array_storage(test_file_name, 2, "wb+", dtype_size_bytes=a[0][0].dtype, num_encodings_types=3, size_encodings=2)

for line in a:
    memmap.save_line_sequential(line)
memmap.save_instances_positions()
memmap.close_file()

In [108]:
memmap = Variable_ELMO_array_storage(test_file_name, 2, "rb", dtype_size_bytes=a[0][0].dtype, num_encodings_types=3, size_encodings=2)

In [109]:
print(memmap.load_line_at_index(1))

here
[array([[[0.41212383, 0.96545007],
        [0.0546776 , 0.7107712 ],
        [0.5505881 , 0.556125  ]],

       [[0.19634092, 0.6540999 ],
        [0.17051387, 0.61365517],
        [0.24236891, 0.17776813]],

       [[0.15770505, 0.56735433],
        [0.19640796, 0.84603931],
        [0.66754472, 0.68189391]]]), array([[[0.52480888, 0.25467228],
        [0.18592735, 0.42112653],
        [0.66452649, 0.74938758],
        [0.66649422, 0.87228193],
        [0.07284315, 0.57713035]],

       [[0.2587362 , 0.07700945],
        [0.01027407, 0.3965815 ],
        [0.22326667, 0.89547983],
        [0.34793616, 0.39370783],
        [0.62556668, 0.82839823]],

       [[0.03790781, 0.9923825 ],
        [0.67404895, 0.81326395],
        [0.85375239, 0.10947917],
        [0.85764065, 0.98802673],
        [0.15409629, 0.86763861]]])]


In [105]:
memmap.close_file()

In [1]:
def file_len(fname):
    with open(fname, "r", encoding='utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [2]:
def tokenize_string(s):
    tokenized_string = []
    split_element = s.split(' ')
    for k, word in enumerate(split_element):
        tokenized_string += nltk.tokenize.word_tokenize(word)
    return tokenized_string

In [21]:
split = train

mnli_split_len = file_len(split)
print(mnli_split_len)

392703


In [19]:
counter = 0
with open(split, "r", encoding='utf-8') as f:
    for l_num, line in enumerate(f):
        if (l_num > 0):
            if (l_num == 1):
                print(line.rstrip('\r\n').split('\t'))
                break
#             if (l_num % min(int(mnli_split_len/3), 250) == 0 or l_num + 1 == mnli_split_len):
#                     print('\r' + str(l_num + 1) + '/' + str(mnli_split_len), end="", flush=True)

#             line_temp = line.rstrip('\r\n').split('\t')
#             if (line_temp[0] != 'contradiction' and line_temp[0] != 'neutral'  and line_temp[0] != 'entailment'):
#                 counter += 1
                
# print("\n\n" + str(counter))

['neutral', '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )', '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )', '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))', '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))', 'Conceptually cream skimming has two basic dimensions - product and geography.', 'Product and geography are what make cream skimming work. ', '31193', '31193n', 'government', 'neutral', '', '', '', '']


In [ ]:
p_data = [] # Propositions
h_data = [] # Hypothesis
labels = [] # Labels

with open(split, "r", encoding='utf-8') as f:
    for l_num, line in enumerate(f):
        
        line_temp = line.rstrip('\r\n').split('\t')
        
        if (l_num > 0 and (line_temp[0] == 'contradiction' or line_temp[0] == 'neutral' or line_temp[0] == 'entailment')):
            if (l_num % min(int(mnli_split_len/3), 250) == 0 or l_num + 1 == mnli_split_len):
                    print('\r' + str(l_num + 1) + '/' + str(mnli_split_len), end="", flush=True)

            labels.append(line_temp[0])
            p_data.append(tokenize_string(line_temp[5]))
            h_data.append(tokenize_string(line_temp[6]))


p_embeddings = np.array(elmo.batch_to_embeddings(p_data))

355501/392703

In [ ]:
print(p_embeddings.shape)

## Memory Maps testing

In [3]:
a = np.random.rand(100000000, 3, 3)

In [4]:
print(a[1])
print(a[1327847])

[[0.18684146 0.48278754 0.41862899]
 [0.94090055 0.23783645 0.43103484]
 [0.53371374 0.16162614 0.63580051]]
[[0.31235169 0.24899142 0.3077785 ]
 [0.25172037 0.09333479 0.07135843]
 [0.71446911 0.29686622 0.11627533]]


In [5]:
b = np.memmap(path_to_test, dtype='float64', mode='w+', shape=a.shape)

b[:] = a[:]
del b
del a

In [6]:
newb = np.memmap(path_to_test, dtype='float64', mode='r', shape=(100000000, 3, 3))#, offset=9*8)

In [7]:
newb.shape

(100000000, 3, 3)

In [12]:
test = np.array(newb[1:4])
print (test)

[[[0.18684146 0.48278754 0.41862899]
  [0.94090055 0.23783645 0.43103484]
  [0.53371374 0.16162614 0.63580051]]

 [[0.94837605 0.02247012 0.27250388]
  [0.79709503 0.99217476 0.87264656]
  [0.04750585 0.63639476 0.75983353]]

 [[0.47735076 0.60599367 0.49551718]
  [0.26734756 0.07261834 0.96984787]
  [0.6244175  0.11239189 0.24127904]]]


In [13]:
t = torch.from_numpy(np.array(newb))

In [14]:
print(t[1])
print(t[1327847])

tensor([[0.1868, 0.4828, 0.4186],
        [0.9409, 0.2378, 0.4310],
        [0.5337, 0.1616, 0.6358]], dtype=torch.float64)
tensor([[0.3124, 0.2490, 0.3078],
        [0.2517, 0.0933, 0.0714],
        [0.7145, 0.2969, 0.1163]], dtype=torch.float64)


In [40]:
del t

In [10]:
del newb

### Create dataset

In [7]:
experiment_types = ['normal', 'zero_shot', 'few_shot']
dataset_types = ['train', 'val', 'test']
new_data_path = "../datasets/Data/Ours/mini_test/"

type_determiner_tokens = ["SUBJECT_ENTITY", "ate", "an", "OBJECT_ENTITY", "for", "breakfast"]
type_determiner_embeddings = elmo.embed_sentence(type_determiner_tokens)
print(type_determiner_embeddings.dtype)

float32


In [25]:
time_encodings = []
time_writing = []
for e_t_num, experiment_type in enumerate(experiment_types):
    for d_t_num, dataset_type in enumerate(dataset_types):

        if (experiment_type == 'normal' and dataset_type == 'train'):
            continue

        file_path = new_data_path + experiment_type + '/' + dataset_type
        f_len = file_len(file_path + '.txt')

        memmap = Variable_ELMO_array_storage(file_path, 1, "wb+", dtype_size_bytes=type_determiner_embeddings.dtype, num_encodings_types=3, size_encodings=1024)

        id_labels_map = []
        labels_id_map = {}
        labels = []

        with open(file_path + '.txt', "r", encoding='utf-8') as f:
            for l_num, line in enumerate(f):
                if (l_num % min(1, 250) == 0 or l_num + 1 == f_len):#int(f_len/3)
                    print('\r' + str(l_num + 1) + '/' + str(f_len), end="", flush=True)

                line_temp = line.rstrip('\r\n').split('\t')

                if (line_temp[0] not in labels_id_map):
                    id_labels_map.append(line_temp[0])
                    labels_id_map[line_temp[0]] = len(id_labels_map) - 1

                labels.append(labels_id_map[line_temp[0]])
                start = time.time()
                encodings = elmo.embed_sentence(tokenize_string(line_temp[1]))
                time_encodings.append(time.time() - start)
                start = time.time()
                memmap.save_line_sequential(encodings)
                time_writing.append(time.time() - start)

        memmap.save_instances_positions()
        memmap.close_file()

        with open(file_path + '.lbs', "wb+") as f:
            pickle.dump(id_labels_map, f)
            pickle.dump(labels_id_map, f)
            pickle.dump(labels, f)

        print("\n\nAverage time encoding: " + '{:.3f}'.format(np.average(np.array(time_encodings))) + "s.")
        print("Average time writing: " + '{:.3f}'.format(np.average(np.array(time_writing))) + "s.\n\n")

600/600

Average time encoding: 0.496s.
Average time writing: 0.103s.


1200/1200

Average time encoding: 0.484s.
Average time writing: 0.104s.


1200/1200

Average time encoding: 0.477s.
Average time writing: 0.103s.


100/100

Average time encoding: 0.475s.
Average time writing: 0.102s.


200/200

Average time encoding: 0.476s.
Average time writing: 0.102s.


1240/1240

Average time encoding: 0.471s.
Average time writing: 0.100s.


100/100

Average time encoding: 0.472s.
Average time writing: 0.101s.


200/200

Average time encoding: 0.473s.
Average time writing: 0.101s.




In [19]:
with open(file_path + '.lbs', "wb+") as f:
            pickle.dump(id_labels_map, f)
            pickle.dump(labels_id_map, f)
            pickle.dump(labels, f)

In [20]:
with open(file_path + '.lbs', "rb") as f:
            id_labels_map = pickle.load(f)
            labels_id_map = pickle.load(f)
            labels = pickle.load(f)

In [21]:
print(len(id_labels_map))
print(len(labels_id_map))
print(id_labels_map[0], labels_id_map[id_labels_map[0]])
print(len(labels))

120
120
IUCN conservation status 0
1800
